In [1]:
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import time

In [9]:
service = Service(ChromeDriverManager().install())
robo = webdriver.Chrome(service=service)
url_va = robo.get("https://veiculos.ssinformatica.net/CMVarzeaAlegre/")

In [10]:
robo.find_element(By.XPATH, '//*[@id="UserName"]').clear()
robo.find_element(By.XPATH, '//*[@id="Password"]').clear()
robo.find_element(By.XPATH, '//*[@id="UserName"]').send_keys('seu_usuario@exemplo.com')
robo.find_element(By.XPATH, '//*[@id="Password"]').send_keys('sua_senha')
robo.find_element(By.XPATH, '/html/body/div[2]/div/form/div[3]/div[5]/div/input').click()

In [4]:
sair = robo.find_element(By.XPATH,'//*[@id="btn-Sair"]').click()

In [11]:
robo.find_element(By.XPATH, '//*[@id="side-menu"]/li[9]/a/span').click()
robo.find_element(By.XPATH, '//*[@id="side-menu"]/li[9]/ul/li[1]/a').click()
robo.find_element(By.XPATH, '//*[@id="toolbar"]/a').click()

In [12]:
caminho_arquivo = "C:/Users/FranklinAlves/.devFk/.eu_dev/automation_selenium/datasets/abastecimento.xlsx"
arquivo = pd.read_excel(caminho_arquivo, sheet_name='mensal_01', dtype={'HORA': str})
arquivo = arquivo.round({'LITROS': 3, 'ODM': 2})
arquivo['DATA'] = arquivo['DATA'].dt.strftime('%d/%m/%Y')
arquivo_str = arquivo.astype(str)
display(arquivo_str)
display(arquivo_str.dtypes)

,DATA,HORA,PLACA,LITROS,ODM,T,CT
0,01/09/2024,0930,PLC-0001,43.251,96349.26,Gasolina,2024
1,02/09/2024,0930,PLC-0001,45.222,96756.26,Gasolina,2024
2,03/09/2024,0830,PLC-0001,47.253,97181.53,Gasolina,2024
3,04/09/2024,1000,PLC-0001,44.529,97582.3,Gasolina,2024
4,05/09/2024,1020,PLC-0001,42.369,97963.62,Gasolina,2024
5,06/09/2024,0830,PLC-0002,41.254,96830.03,Gasolina,2024
6,07/09/2024,0930,PLC-0002,42.057,97166.49,Gasolina,2024
7,08/09/2024,1000,PLC-0002,39.547,97482.86,Gasolina,2024
8,09/09/2024,1100,PLC-0002,37.027,97779.08,Gasolina,2024
9,10/09/2024,1430,PLC-0002,42.658,98120.34,Gasolina,2024


DATA      object
HORA      object
PLACA     object
LITROS    object
ODM       object
T         object
CT        object
dtype: object

In [18]:
for i, data in enumerate(arquivo_str["DATA"]):
    placa = arquivo_str.loc[i, "PLACA"]
    km = arquivo_str.loc[i, "ODM"]
    litros = arquivo_str.loc[i, "LITROS"]
    tipo = arquivo_str.loc[i, "T"]
    contrato = arquivo_str.loc[i, "CT"]
    hora = arquivo_str.loc[i, "HORA"]

    campo_data = robo.find_element(By.XPATH, '//*[@id="DataAbastecimento"]')
    campo_data.clear()
    campo_data.click()
    robo.execute_script("arguments[0].setSelectionRange(0, 0);", campo_data)
    campo_data.send_keys(data, hora)

    campo_placa = robo.find_element(By.XPATH, '//*[@id="Veiculo"]')
    campo_placa.clear()
    campo_placa.send_keys(placa)

    try:
        sugestao_placa = WebDriverWait(robo, 10).until(
            EC.visibility_of_element_located((By.XPATH, f"//li[contains(., '{placa}')]"))
        )
        campo_placa.send_keys(Keys.ENTER)
        
    except Exception as e:
        print(f"Erro ao aguardar pela sugestão da placa: {e}")

    campo_km = robo.find_element(By.XPATH, '//*[@id="QuilometragemHorimetroAtual"]')
    campo_km.send_keys(km)

    campo_contrato = robo.find_element(By.XPATH, '//*[@id="NumeroContrato"]')
    campo_contrato.clear()
    campo_contrato.send_keys(contrato)

    try:
        sugestao_contrato = WebDriverWait(robo, 10).until(
            EC.visibility_of_element_located((By.XPATH, f"//li[contains(., '{contrato}')]"))
        )
        campo_contrato.send_keys(Keys.ENTER)

    except Exception as e:
        print(f"Erro ao aguardar pela sugestão do contrato: {e}")

    robo.find_element(By.XPATH, '//*[@id="btnSalvarContinuarInformandoItens"]').click()

    try:
        botao_ok = WebDriverWait(robo, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/div[6]/div[7]/button[2]'))
        )
        botao_ok.click()

    except Exception as e:
        print(f"Erro ao aguardar pelo botão OK de confirmação: {e}")

    campo_combustivel = robo.find_element(By.XPATH, '//*[@id="Produto"]')
    campo_combustivel.send_keys(tipo)

    try:
        sugestao_combustivel = WebDriverWait(robo, 10).until(
            EC.visibility_of_element_located((By.XPATH, f"//li[contains(., '{tipo}')]"))
        )
        campo_combustivel.send_keys(Keys.ENTER)

    except Exception as e:
        print(f"Erro ao aguardar pela sugestão do tipo de combustível: {e}")

    campo_litros = robo.find_element(By.XPATH, '//*[@id="Quantidade"]')
    campo_litros.send_keys(litros)

    salvar = robo.find_element(By.XPATH, '//*[@id="btnSalvarItemAbastecimento"]').click()
    time.sleep(0.5)
    
    encerrar = robo.find_element(By.XPATH, '//*[@id="btnFinalizarAbastecimento"]').click()

    try:
        botao_ok = WebDriverWait(robo, 15).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/div[6]/div[7]/button[2]'))
        )
        time.sleep(1.5)
        botao_ok.click()
        print(f"Abastecimento {data} - {placa} Finalizado!")

    except Exception as e:
        print(f"Erro ao aguardar pelo botão OK de finalização: {e}")

    time.sleep(2)

    robo.find_element(By.XPATH, '//*[@id="toolbar"]/a').click()
print(f'Abastecimentos referente ao consumo do mês de SETEMBRO finalizado!')

robo.find_element(By.XPATH, '//*[@id="side-menu"]/li[9]/ul/li[1]/a').click()


Abastecimento 01/09/2024 - PLC-0001 Finalizado!
Abastecimento 02/09/2024 - PLC-0001 Finalizado!
Abastecimento 03/09/2024 - PLC-0001 Finalizado!
Abastecimento 04/09/2024 - PLC-0001 Finalizado!
Abastecimento 05/09/2024 - PLC-0001 Finalizado!
Abastecimento 06/09/2024 - PLC-0002 Finalizado!
Abastecimento 07/09/2024 - PLC-0002 Finalizado!
Abastecimento 08/09/2024 - PLC-0002 Finalizado!
Abastecimento 09/09/2024 - PLC-0002 Finalizado!
Abastecimento 10/09/2024 - PLC-0002 Finalizado!
Abastecimentos referente ao consumo do mês de SETEMBRO finalizado!
